In [1]:
from transformers import AutoFeatureExtractor, ASTForAudioClassification
from datasets import load_dataset
import torch
import os
import pandas as pd 
import numpy as np 

import torch.nn as nn
import torch.optim as optim

import warnings
warnings.filterwarnings("ignore")

C:\Users\zhuld\anaconda3\envs\pytorch_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
path = 'C:\\Users\\zhuld\\Dropbox\\EAV'
print(os.listdir(path))

['.ipynb_checkpoints', 'desktop.ini', 'GitHub_Codes', 'meta_data.csv', 'questionnaire.xlsx', 'subject1', 'subject10', 'subject11', 'subject12', 'subject13', 'subject14', 'subject15', 'subject16', 'subject17', 'subject18', 'subject19', 'subject2', 'subject20', 'subject21', 'subject22', 'subject23', 'subject24', 'subject25', 'subject26', 'subject27', 'subject28', 'subject29', 'subject3', 'subject30', 'subject31', 'subject32', 'subject33', 'subject34', 'subject35', 'subject36', 'subject37', 'subject38', 'subject39', 'subject4', 'subject40', 'subject41', 'subject42', 'subject5', 'subject6', 'subject7', 'subject8', 'subject9', 'temp_subjects.csv']


In [4]:
cnt = 0 
for sub in os.listdir(path):
    if sub[:7] == 'subject':
        for i in os.listdir(path+'/'+sub+'/Audio'):
            print(i)
            cnt += 1 

print(cnt)

002_Trial_02_Speaking_Neutral_Aud.wav
004_Trial_02_Speaking_Anger_aud.wav
006_Trial_02_Speaking_Anger_aud.wav
008_Trial_02_Speaking_Anger_aud.wav
010_Trial_02_Speaking_Anger_aud.wav
012_Trial_02_Speaking_Neutral_aud.wav
014_Trial_02_Speaking_Anger_aud.wav
016_Trial_02_Speaking_Anger_aud.wav
018_Trial_02_Speaking_Anger_aud.wav
020_Trial_02_Speaking_Anger_aud.wav
022_Trial_02_Speaking_Neutral_aud.wav
024_Trial_02_Speaking_Calmness_aud.wav
026_Trial_02_Speaking_Calmness_aud.wav
028_Trial_02_Speaking_Calmness_aud.wav
030_Trial_02_Speaking_Calmness_aud.wav
032_Trial_02_Speaking_Neutral_aud.wav
034_Trial_02_Speaking_Calmness_aud.wav
036_Trial_02_Speaking_Calmness_aud.wav
038_Trial_02_Speaking_Calmness_aud.wav
040_Trial_02_Speaking_Calmness_aud.wav
042_Trial_04_Speaking_Neutral_aud.wav
044_Trial_04_Speaking_Sadness_aud.wav
046_Trial_04_Speaking_Sadness_aud.wav
048_Trial_04_Speaking_Sadness_aud.wav
050_Trial_04_Speaking_Sadness_aud.wav
052_Trial_04_Speaking_Neutral_aud.wav
054_Trial_04_Speakin

In [5]:
file_emotion = []
file_path = []
subjects = []


for sub in os.listdir(path):
    if sub[:7] == 'subject':
        for i in os.listdir(path+'/'+sub+'/Audio'):
            emotion = i.split('_')[4]
            file_emotion.append(emotion)
            file_path.append(path+'/'+sub+'/Audio/'+i)
            subjects.append(sub)

emotion_df = pd.DataFrame(file_emotion, columns=['Emotion'])
path_df = pd.DataFrame(file_path, columns=['Path'])
subjects_df = pd.DataFrame(subjects, columns=['Subject']) 
df = pd.concat([subjects_df, emotion_df, path_df], axis=1)
df

,Subject,Emotion,Path
0,subject1,Neutral,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/002_...
1,subject1,Anger,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/004_...
2,subject1,Anger,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/006_...
3,subject1,Anger,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/008_...
4,subject1,Anger,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/010_...
...,...,...,...
4195,subject9,Neutral,C:\Users\zhuld\Dropbox\EAV/subject9/Audio/192_...
4196,subject9,Sadness,C:\Users\zhuld\Dropbox\EAV/subject9/Audio/194_...
4197,subject9,Sadness,C:\Users\zhuld\Dropbox\EAV/subject9/Audio/196_...
4198,subject9,Sadness,C:\Users\zhuld\Dropbox\EAV/subject9/Audio/198_...


### sort subjects 

In [6]:
import re 

def extract_number(s): 
    return int(re.search(r'\d+', s).group())

df['numeric_part'] = df['Subject'].apply(extract_number)
df.sort_values('numeric_part', inplace=True)
df.drop('numeric_part', axis=1, inplace=True)

df

,Subject,Emotion,Path
0,subject1,Neutral,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/002_...
72,subject1,Anger,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/146_...
71,subject1,Anger,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/144_...
70,subject1,Neutral,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/142_...
69,subject1,Anger,C:\Users\zhuld\Dropbox\EAV/subject1/Audio/140_...
...,...,...,...
3628,subject42,Sadness,C:\Users\zhuld\Dropbox\EAV/subject42/Audio/058...
3627,subject42,Sadness,C:\Users\zhuld\Dropbox\EAV/subject42/Audio/056...
3626,subject42,Sadness,C:\Users\zhuld\Dropbox\EAV/subject42/Audio/054...
3636,subject42,Anger,C:\Users\zhuld\Dropbox\EAV/subject42/Audio/074...


In [7]:
from transformers import ASTFeatureExtractor

feature_extractor = ASTFeatureExtractor()

### example of how to load audio data and resample it to 16000, ASTFeature extractor works with 16000 sampling rate 

In [8]:
import torchaudio
from torchaudio.transforms import Resample

filepath = df.iloc[0].Path

waveform, sampling_rate = torchaudio.load(filepath)
#waveform = waveform.squeeze().numpy()

print("Original sampling rate is: ", sampling_rate)

target_sampling_rate = 16000
resampler = Resample(orig_freq=sampling_rate, new_freq=target_sampling_rate)
resampled_waveform = resampler(waveform)

waveform = waveform.squeeze().numpy()
resampled_waveform = resampled_waveform.squeeze().numpy()

print("Original waveform shape:", waveform.shape)
print("Resampled waveform shape: ", resampled_waveform.shape)

Original sampling rate is:  44100
Original waveform shape: (926100,)
Resampled waveform shape:  (336000,)


In [93]:
from transformers import AutoModelForAudioClassification

model = AutoModelForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

In [10]:
df.Emotion.unique()

array(['Neutral', 'Anger', 'Sadness', 'Happiness', 'Calmness'],
      dtype=object)

In [17]:
target_sampling_rate = 16000
test_features_list = []
train_features_list = []
test_labels_list = [] 
train_labels_list = []  

num_classes = 5

# Mapping of string emotions to integer labels
emotion_to_index = {
    'Happiness': 0,
    'Neutral': 1,
    'Sadness': 2,
    'Anger': 3,
    'Calmness': 4
}

# inverse mapping 
index_to_emotion = {v: k for k, v in emotion_to_index.items()}

# dict to store number of emotions stored 
em_counts = {emotion: 0 for emotion in emotion_to_index.keys()}

for path, sub, em in zip(df.Path, df.Subject, df.Emotion):
    if sub == 'subject1':
        waveform, sampling_rate = torchaudio.load(path)
    
        resampler = Resample(orig_freq=sampling_rate, new_freq=target_sampling_rate)
        resampled_waveform = resampler(waveform)
            
        waveform = waveform.squeeze().numpy()
        resampled_waveform = resampled_waveform.squeeze().numpy()
    
        inputs = feature_extractor(resampled_waveform, sampling_rate=target_sampling_rate, padding="max_length", return_tensors="pt")
        input_values = inputs.input_values
    
        label_idx = emotion_to_index[em]
        label = emotion_to_index[em]

        if em_counts[em] < 10: 
            train_features_list.append(input_values)
            train_labels_list.append(label)
        else: 
            test_features_list.append(input_values)
            test_labels_list.append(label)

        em_counts[em] += 1 
        
    else:
        continue

In [20]:
len(train_features_list), len(test_features_list), len(train_labels_list), len(test_labels_list)

(50, 50, 50, 50)

In [22]:
em_counts

{'Happiness': 20, 'Neutral': 20, 'Sadness': 20, 'Anger': 20, 'Calmness': 20}

In [25]:
for i in range(5):
    print(f'Number of {index_to_emotion[i]} emotions is {train_labels_list.count(i)} for training set')
    print(f'Number of {index_to_emotion[i]} emotions is {test_labels_list.count(i)} for test set')

Number of Happiness emotions is 10 for training set
Number of Happiness emotions is 10 for test set
Number of Neutral emotions is 10 for training set
Number of Neutral emotions is 10 for test set
Number of Sadness emotions is 10 for training set
Number of Sadness emotions is 10 for test set
Number of Anger emotions is 10 for training set
Number of Anger emotions is 10 for test set
Number of Calmness emotions is 10 for training set
Number of Calmness emotions is 10 for test set


In [26]:
test_features = torch.cat(test_features_list, dim=0)
train_features = torch.cat(train_features_list, dim=0)
test_features.shape, train_features.shape

(torch.Size([50, 1024, 128]), torch.Size([50, 1024, 128]))

In [27]:
test_labels = torch.Tensor(test_labels_list)
train_labels = torch.Tensor(train_labels_list)
test_labels.shape, train_labels.shape

(torch.Size([50]), torch.Size([50]))

In [28]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(train_features, train_labels)
test_dataset = TensorDataset(test_features, test_labels)

In [29]:
from torch.utils.data import DataLoader

loader_train = DataLoader(train_dataset, batch_size=8, shuffle=True)
loader_test = DataLoader(test_dataset, batch_size=8, shuffle=True)

In [30]:
for batch_idx, (features, labels) in enumerate(loader_train):
    print(features.shape, labels.shape)
    break

torch.Size([8, 1024, 128]) torch.Size([8])


In [31]:
model 

ASTForAudioClassification(
  (audio_spectrogram_transformer): ASTModel(
    (embeddings): ASTEmbeddings(
      (patch_embeddings): ASTPatchEmbeddings(
        (projection): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ASTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ASTLayer(
          (attention): ASTAttention(
            (attention): ASTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ASTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ASTIntermediate(
            (de

## with segmentation

In [47]:
def divide_into_segments (data, sr, seg_dur_seconds = 5):
    segment_length = sr * seg_dur_seconds
    num_sections = int(np.floor(len(data) / segment_length))
    split = []
    for i in range(num_sections):
        t = data[i * segment_length: (i + 1) * segment_length]
        split.append(t)
    return split 

In [58]:
filepath = df.iloc[0].Path

waveform, sampling_rate = torchaudio.load(filepath)
#waveform = waveform.squeeze().numpy()

print("Original sampling rate is: ", sampling_rate)

target_sampling_rate = 16000
resampler = Resample(orig_freq=sampling_rate, new_freq=target_sampling_rate)
resampled_waveform = resampler(waveform)

waveform = waveform.squeeze().numpy()
resampled_waveform = resampled_waveform.squeeze().numpy()

print("Original waveform shape:", waveform.shape)
print("Resampled waveform shape: ", resampled_waveform.shape)

inputs = feature_extractor(resampled_waveform, sampling_rate=target_sampling_rate, padding="max_length", return_tensors="pt")
input_values = inputs.input_values
print(input_values.shape)

Original sampling rate is:  44100
Original waveform shape: (926100,)
Resampled waveform shape:  (336000,)
torch.Size([1, 1024, 128])


In [51]:
split = divide_into_segments(resampled_waveform, target_sampling_rate)
split

[array([-0.00420913, -0.00697077, -0.00660943, ..., -0.01008654,
        -0.00871788, -0.00857039], dtype=float32),
 array([-0.00678314, -0.00634588, -0.00610846, ...,  0.00211733,
         0.0020789 ,  0.00183299], dtype=float32),
 array([ 0.00194718,  0.00215848,  0.00226653, ..., -0.00013027,
         0.00014251,  0.00034362], dtype=float32),
 array([-0.00026832, -0.00019455, -0.0003    , ..., -0.0017999 ,
        -0.00189009, -0.00193078], dtype=float32)]

In [57]:
for i in range(4):
    print(split[i].shape)
    inputs = feature_extractor(split[i], sampling_rate=target_sampling_rate, padding="max_length", return_tensors="pt")
    input_values = inputs.input_values
    print(input_values.shape)

(80000,)
torch.Size([1, 1024, 128])
(80000,)
torch.Size([1, 1024, 128])
(80000,)
torch.Size([1, 1024, 128])
(80000,)
torch.Size([1, 1024, 128])


In [94]:
from transformers import AutoModelForAudioClassification

model = AutoModelForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

### Emotion recognition for subject 2 only 

In [95]:
target_sampling_rate = 16000
test_features_list = []
train_features_list = []
test_labels_list = [] 
train_labels_list = []  

num_classes = 5

# Mapping of string emotions to integer labels
emotion_to_index = {
    'Happiness': 0,
    'Neutral': 1,
    'Sadness': 2,
    'Anger': 3,
    'Calmness': 4
}

# inverse mapping 
index_to_emotion = {v: k for k, v in emotion_to_index.items()}

# dict to store number of emotions stored 
em_counts = {emotion: 0 for emotion in emotion_to_index.keys()}

for path, sub, em in zip(df.Path, df.Subject, df.Emotion):
    if sub == 'subject2':
        waveform, sampling_rate = torchaudio.load(path)
    
        resampler = Resample(orig_freq=sampling_rate, new_freq=target_sampling_rate)
        resampled_waveform = resampler(waveform)
            
        waveform = waveform.squeeze().numpy()
        resampled_waveform = resampled_waveform.squeeze().numpy()

        split = divide_into_segments(resampled_waveform, target_sampling_rate) # list of 4 torch tensors 

        for i in range(len(split)):
            inputs = feature_extractor(split[i], sampling_rate=target_sampling_rate, padding='max_length', return_tensors='pt')
            input_values = inputs.input_values

            label_idx = emotion_to_index[em]
            label = emotion_to_index[em]

            if em_counts[em] < 10: 
                train_features_list.append(input_values)
                train_labels_list.append(label)
            else:
                test_features_list.append(input_values)
                test_labels_list.append(label)

        em_counts[em] += 1 
        
    else:
        continue

In [77]:
len(train_features_list), len(test_features_list), len(train_labels_list), len(test_labels_list)

(200, 200, 200, 200)

In [96]:
for i in range(5):
    print(f'Number of {index_to_emotion[i]} emotions is {train_labels_list.count(i)} for training set')
    print(f'Number of {index_to_emotion[i]} emotions is {test_labels_list.count(i)} for test set')

Number of Happiness emotions is 40 for training set
Number of Happiness emotions is 40 for test set
Number of Neutral emotions is 40 for training set
Number of Neutral emotions is 40 for test set
Number of Sadness emotions is 40 for training set
Number of Sadness emotions is 40 for test set
Number of Anger emotions is 40 for training set
Number of Anger emotions is 40 for test set
Number of Calmness emotions is 40 for training set
Number of Calmness emotions is 40 for test set


In [97]:
test_features = torch.cat(test_features_list, dim=0)
train_features = torch.cat(train_features_list, dim=0)
test_features.shape, train_features.shape

(torch.Size([200, 1024, 128]), torch.Size([200, 1024, 128]))

In [98]:
test_labels = torch.Tensor(test_labels_list)
train_labels = torch.Tensor(train_labels_list)
test_labels.shape, train_labels.shape

(torch.Size([200]), torch.Size([200]))

In [99]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(train_features, train_labels)
test_dataset = TensorDataset(test_features, test_labels)

In [100]:
from torch.utils.data import DataLoader

loader_train = DataLoader(train_dataset, batch_size=8, shuffle=True)
loader_test = DataLoader(test_dataset, batch_size=8, shuffle=True)

In [101]:
model.classifier.dense = torch.nn.Linear(model.classifier.dense.in_features, 5)

# Freeze all the layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the classifier layer 
for param in model.classifier.dense.parameters():
    param.requires_grad = True

In [102]:
def add_weight_decay(model, weight_decay=1e-5, skip_list=()):
    decay = []
    no_decay = []
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue  # frozen weights
        if len(param.shape) == 1 or name.endswith(".bias") or name in skip_list:
            no_decay.append(param)
        else:
            decay.append(param)
    return [
        {'params': no_decay, 'weight_decay': 0.},
        {'params': decay, 'weight_decay': weight_decay}]

# weight_decay
skip = {}
if hasattr(model, 'no_weight_decay'):
    skip = model.no_weight_decay()

parameters = add_weight_decay(model, 0.0001, skip)
weight_decay = 0.

optimizer = optim.AdamW(parameters)
loss_fn = nn.CrossEntropyLoss()

In [103]:
from IPython.display import HTML, display

def progress(value, max=100):
    return HTML("""

            {value}

    """.format(value=value, max=max))

In [104]:
train_len, test_len = len(train_dataset), len(test_dataset)

def eval_corona(model, loader_val):
  model.eval()

  preds = []
  with torch.no_grad():
    for x, t in loader_val:
      x, t = x.to(device), t.long().to(device) # comment this line if use option1 dataloader
      logits = model(x).logits
      preds.append(torch.sum(torch.max(logits, dim=1)[1] == t))

  return sum(preds) / test_len

In [105]:
iters = 20
train_len, test_len = len(train_dataset), len(test_dataset)
pb_train = display(progress(0, iters), display_id=True)
losses = [[]]
accus_train = [[]]
accus_val = []

model = model.to(device)

for i in range(iters):
  for x, t in loader_train:
    x, t = x.to(device), t.long().to(device)
    #x, t = x.to(device), t.to(device).long() # comment this line if use option1 dataloader
    logits = model(x).logits
    #print(logits)
    # print(logits.mean())
    loss = loss_fn(logits, t)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    losses[-1].append(loss.item())

    with torch.no_grad():
      accus_train[-1].append(torch.sum(torch.max(logits, dim=1)[1] == t))
      #print(torch.max(logits, dim=1)[1])
    # break

  losses[-1] = sum(losses[-1])
  losses.append([])
  accus_train[-1] = sum(accus_train[-1]) / train_len
  accus_train.append([])

  # step LR for next epoch
  #lr_scheduler.step(i + 1)

  accus_val.append(eval_corona(model, loader_test))
  model.train()

  pb_train.update(progress(i, iters))
  print(f'\r[Step {i+1}] loss={loss:.2e} '
        f'\r[Step {i+1}] loss={losses[-2]:.2e} '
        f'train accu={accus_train[-2]:.2%} '
        f'validation accu={accus_val[-1]:.2%} '
        # , end=''
        , flush=True)
  # break

[Step 1] loss=3.98e+01 train accu=25.50% validation accu=44.50% 
[Step 2] loss=3.15e+01 train accu=52.00% validation accu=48.00% 
[Step 3] loss=2.73e+01 train accu=63.50% validation accu=58.00% 
[Step 4] loss=2.43e+01 train accu=72.00% validation accu=56.00% 
[Step 5] loss=2.23e+01 train accu=69.00% validation accu=48.50% 
[Step 6] loss=2.07e+01 train accu=73.00% validation accu=59.00% 
[Step 7] loss=1.90e+01 train accu=76.00% validation accu=60.50% 
[Step 8] loss=1.70e+01 train accu=84.00% validation accu=56.00% 
[Step 9] loss=1.60e+01 train accu=87.00% validation accu=60.50% 
[Step 10] loss=1.50e+01 train accu=85.00% validation accu=59.50% 
[Step 11] loss=1.47e+01 train accu=84.50% validation accu=60.50% 
[Step 12] loss=1.36e+01 train accu=88.00% validation accu=60.50% 
[Step 13] loss=1.29e+01 train accu=88.50% validation accu=60.00% 
[Step 14] loss=1.22e+01 train accu=85.50% validation accu=64.50% 
[Step 15] loss=1.16e+01 train accu=90.50% validation accu=63.50% 
[Step 16] loss=1.07

In [90]:
model = torch.load(save_path)
model = model.to(device)
model.eval()

correct = 0
total = 0

with torch.no_grad():
  for x, t in loader_test:
    x, t = x.to(device), t.long().to(device)

    logits = model(x).logits
    predicted = torch.max(logits, dim=1)[1]

    total += t.size(0)
    correct += (predicted==t).sum().item()

finetune_acc = correct/total
print(f'Test set accuracy: {finetune_acc*100:.3f}%')

Test set accuracy: 65.000%


In [92]:
subjects = [f'subject{i}' for i in range(1, 43)]

In [111]:
sub_accuracies = {subj: 0 for subj in subjects}

for subj in subjects:
    if subj == 'subject1' or subj == 'subject2':
        continue

    model = AutoModelForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

    target_sampling_rate = 16000
    test_features_list = []
    train_features_list = []
    test_labels_list = [] 
    train_labels_list = []  
    
    num_classes = 5
    
    # dict to store number of emotions stored 
    em_counts = {emotion: 0 for emotion in emotion_to_index.keys()}
    
    for path, sub, em in zip(df.Path, df.Subject, df.Emotion):
        if sub == subj: # subj iterates though each subject 
            waveform, sampling_rate = torchaudio.load(path)
        
            resampler = Resample(orig_freq=sampling_rate, new_freq=target_sampling_rate)
            resampled_waveform = resampler(waveform)
                
            waveform = waveform.squeeze().numpy()
            resampled_waveform = resampled_waveform.squeeze().numpy()
    
            split = divide_into_segments(resampled_waveform, target_sampling_rate) # list of 4 torch tensors 
    
            for i in range(len(split)):
                inputs = feature_extractor(split[i], sampling_rate=target_sampling_rate, padding='max_length', return_tensors='pt')
                input_values = inputs.input_values
    
                label_idx = emotion_to_index[em]
                label = emotion_to_index[em]
    
                if em_counts[em] < 10: 
                    train_features_list.append(input_values)
                    train_labels_list.append(label)
                else:
                    test_features_list.append(input_values)
                    test_labels_list.append(label)
    
            em_counts[em] += 1 
            
        else:
            continue

    test_features = torch.cat(test_features_list, dim=0)
    train_features = torch.cat(train_features_list, dim=0)
    test_labels = torch.Tensor(test_labels_list)
    train_labels = torch.Tensor(train_labels_list)

    train_dataset = TensorDataset(train_features, train_labels)
    test_dataset = TensorDataset(test_features, test_labels)
    loader_train = DataLoader(train_dataset, batch_size=8, shuffle=True)
    loader_test = DataLoader(test_dataset, batch_size=8, shuffle=True)

    model.classifier.dense = torch.nn.Linear(model.classifier.dense.in_features, 5)
    
    for param in model.parameters():
        param.requires_grad = False
    
    for param in model.classifier.dense.parameters():
        param.requires_grad = True

    # weight_decay
    skip = {}
    if hasattr(model, 'no_weight_decay'):
        skip = model.no_weight_decay()
    
    parameters = add_weight_decay(model, 0.0001, skip)
    weight_decay = 0.
    
    optimizer = optim.AdamW(parameters)
    loss_fn = nn.CrossEntropyLoss()

    iters = 20
    train_len, test_len = len(train_dataset), len(test_dataset)
    pb_train = display(progress(0, iters), display_id=True)
    losses = [[]]
    accus_train = [[]]
    accus_val = []
    
    model = model.to(device)
    
    max_val = -1
    save_path = f"audioT{subj}_dep.pth"
    
    for i in range(iters):
      for x, t in loader_train:
        x, t = x.to(device), t.long().to(device)
        #x, t = x.to(device), t.to(device).long() # comment this line if use option1 dataloader
        logits = model(x).logits
        #print(logits)
        # print(logits.mean())
        loss = loss_fn(logits, t)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses[-1].append(loss.item())
    
        with torch.no_grad():
          accus_train[-1].append(torch.sum(torch.max(logits, dim=1)[1] == t))
          #print(torch.max(logits, dim=1)[1])
        # break
    
      losses[-1] = sum(losses[-1])
      losses.append([])
      accus_train[-1] = sum(accus_train[-1]) / train_len
      accus_train.append([])
    
      # step LR for next epoch
      #lr_scheduler.step(i + 1)
    
      accus_val.append(eval_corona(model, loader_test))
      cur_val_acc = accus_val[-1]
      if cur_val_acc > max_val:
        max_val = cur_val_acc
        torch.save(model, save_path)
 
      model.train()
    
      pb_train.update(progress(i, iters))
      print(f'\r[Step {i+1}] loss={loss:.2e} '
            f'\r[Step {i+1}] loss={losses[-2]:.2e} '
            f'train accu={accus_train[-2]:.2%} '
            f'validation accu={accus_val[-1]:.2%} '
            # , end=''
            , flush=True)
      # break

    
    model = torch.load(save_path)
    model = model.to(device)
    model.eval()
    
    correct = 0
    total = 0
    
    with torch.no_grad():
      for x, t in loader_test:
        x, t = x.to(device), t.long().to(device)
    
        logits = model(x).logits
        predicted = torch.max(logits, dim=1)[1]
    
        total += t.size(0)
        correct += (predicted==t).sum().item()
    
    final_acc = correct/total
    print(f'Test set accuracy: {final_acc*100:.3f}%')
    sub_accuracies[subj] = final_acc

[Step 1] loss=4.34e+01 train accu=23.50% validation accu=30.50% 
[Step 2] loss=3.66e+01 train accu=36.00% validation accu=38.00% 
[Step 3] loss=3.26e+01 train accu=48.00% validation accu=42.00% 
[Step 4] loss=3.05e+01 train accu=55.00% validation accu=40.50% 
[Step 5] loss=2.74e+01 train accu=56.00% validation accu=53.50% 
[Step 6] loss=2.67e+01 train accu=57.50% validation accu=41.50% 
[Step 7] loss=2.63e+01 train accu=56.00% validation accu=45.50% 
[Step 8] loss=2.55e+01 train accu=58.50% validation accu=49.00% 
[Step 9] loss=2.39e+01 train accu=62.50% validation accu=47.50% 
[Step 10] loss=2.26e+01 train accu=68.50% validation accu=44.50% 
[Step 11] loss=2.22e+01 train accu=67.50% validation accu=46.50% 
[Step 12] loss=2.09e+01 train accu=72.50% validation accu=45.00% 
[Step 13] loss=2.04e+01 train accu=71.00% validation accu=50.00% 
[Step 14] loss=1.93e+01 train accu=77.00% validation accu=44.50% 
[Step 15] loss=1.86e+01 train accu=75.50% validation accu=48.50% 
[Step 16] loss=1.90

[Step 1] loss=4.09e+01 train accu=27.50% validation accu=36.50% 
[Step 2] loss=3.12e+01 train accu=48.50% validation accu=41.00% 
[Step 3] loss=2.62e+01 train accu=60.50% validation accu=47.50% 
[Step 4] loss=2.35e+01 train accu=68.50% validation accu=50.00% 
[Step 5] loss=2.17e+01 train accu=72.00% validation accu=49.50% 
[Step 6] loss=2.02e+01 train accu=72.00% validation accu=48.50% 
[Step 7] loss=1.93e+01 train accu=71.50% validation accu=47.50% 
[Step 8] loss=1.73e+01 train accu=78.50% validation accu=54.00% 
[Step 9] loss=1.70e+01 train accu=78.50% validation accu=49.50% 
[Step 10] loss=1.65e+01 train accu=80.00% validation accu=57.00% 
[Step 11] loss=1.54e+01 train accu=82.00% validation accu=52.00% 
[Step 12] loss=1.46e+01 train accu=84.00% validation accu=54.00% 
[Step 13] loss=1.31e+01 train accu=87.00% validation accu=58.00% 
[Step 14] loss=1.26e+01 train accu=86.00% validation accu=55.50% 
[Step 15] loss=1.27e+01 train accu=87.50% validation accu=51.00% 
[Step 16] loss=1.21

[Step 1] loss=4.38e+01 train accu=21.00% validation accu=22.50% 
[Step 2] loss=3.75e+01 train accu=37.00% validation accu=35.00% 
[Step 3] loss=3.34e+01 train accu=45.00% validation accu=35.50% 
[Step 4] loss=3.07e+01 train accu=56.00% validation accu=36.00% 
[Step 5] loss=2.97e+01 train accu=52.00% validation accu=35.00% 
[Step 6] loss=2.74e+01 train accu=59.50% validation accu=41.00% 
[Step 7] loss=2.55e+01 train accu=65.00% validation accu=41.50% 
[Step 8] loss=2.57e+01 train accu=62.00% validation accu=42.50% 
[Step 9] loss=2.33e+01 train accu=71.50% validation accu=46.00% 
[Step 10] loss=2.26e+01 train accu=71.00% validation accu=45.00% 
[Step 11] loss=2.31e+01 train accu=65.50% validation accu=42.00% 
[Step 12] loss=2.02e+01 train accu=72.50% validation accu=46.50% 
[Step 13] loss=1.98e+01 train accu=77.00% validation accu=45.00% 
[Step 14] loss=1.92e+01 train accu=76.50% validation accu=42.00% 
[Step 15] loss=1.79e+01 train accu=81.00% validation accu=44.50% 
[Step 16] loss=1.82

[Step 1] loss=4.04e+01 train accu=27.00% validation accu=44.00% 
[Step 2] loss=2.98e+01 train accu=60.00% validation accu=56.00% 
[Step 3] loss=2.47e+01 train accu=64.50% validation accu=47.00% 
[Step 4] loss=2.22e+01 train accu=70.00% validation accu=59.00% 
[Step 5] loss=1.88e+01 train accu=78.50% validation accu=59.50% 
[Step 6] loss=1.72e+01 train accu=76.00% validation accu=69.50% 
[Step 7] loss=1.61e+01 train accu=81.00% validation accu=68.50% 
[Step 8] loss=1.35e+01 train accu=85.50% validation accu=67.00% 
[Step 9] loss=1.25e+01 train accu=85.50% validation accu=68.00% 
[Step 10] loss=1.20e+01 train accu=88.50% validation accu=66.00% 
[Step 11] loss=1.07e+01 train accu=90.50% validation accu=68.00% 
[Step 12] loss=1.04e+01 train accu=89.00% validation accu=66.50% 
[Step 13] loss=9.96e+00 train accu=91.00% validation accu=69.00% 
[Step 14] loss=9.16e+00 train accu=90.00% validation accu=65.50% 
[Step 15] loss=8.76e+00 train accu=92.00% validation accu=68.50% 
[Step 16] loss=8.45

[Step 1] loss=3.96e+01 train accu=26.50% validation accu=50.50% 
[Step 2] loss=3.09e+01 train accu=50.50% validation accu=61.50% 
[Step 3] loss=2.59e+01 train accu=62.50% validation accu=57.00% 
[Step 4] loss=2.36e+01 train accu=64.00% validation accu=70.00% 
[Step 5] loss=2.13e+01 train accu=73.00% validation accu=66.00% 
[Step 6] loss=1.99e+01 train accu=73.00% validation accu=66.00% 
[Step 7] loss=1.89e+01 train accu=75.50% validation accu=70.50% 
[Step 8] loss=1.67e+01 train accu=80.00% validation accu=56.50% 
[Step 9] loss=1.63e+01 train accu=80.50% validation accu=61.50% 
[Step 10] loss=1.61e+01 train accu=80.00% validation accu=64.00% 
[Step 11] loss=1.54e+01 train accu=78.50% validation accu=65.50% 
[Step 12] loss=1.39e+01 train accu=84.50% validation accu=65.00% 
[Step 13] loss=1.33e+01 train accu=85.50% validation accu=63.50% 
[Step 14] loss=1.25e+01 train accu=84.50% validation accu=69.50% 
[Step 15] loss=1.27e+01 train accu=87.50% validation accu=67.00% 
[Step 16] loss=1.17

...

[Step 1] loss=4.13e+01 train accu=28.00% validation accu=33.50% 
[Step 2] loss=3.63e+01 train accu=35.50% validation accu=33.00% 
[Step 3] loss=3.23e+01 train accu=49.00% validation accu=30.50% 
[Step 4] loss=3.01e+01 train accu=52.00% validation accu=38.50% 
[Step 5] loss=2.76e+01 train accu=61.00% validation accu=32.50% 
[Step 6] loss=2.70e+01 train accu=60.00% validation accu=33.50% 
[Step 7] loss=2.61e+01 train accu=61.00% validation accu=33.50% 
[Step 8] loss=2.49e+01 train accu=62.50% validation accu=35.50% 
[Step 9] loss=2.36e+01 train accu=71.50% validation accu=37.50% 
[Step 10] loss=2.28e+01 train accu=71.50% validation accu=36.50% 
[Step 11] loss=2.23e+01 train accu=70.00% validation accu=39.00% 
[Step 12] loss=2.17e+01 train accu=72.00% validation accu=39.00% 
[Step 13] loss=2.06e+01 train accu=78.00% validation accu=37.50% 
[Step 14] loss=1.96e+01 train accu=77.50% validation accu=37.00% 
[Step 15] loss=1.97e+01 train accu=75.50% validation accu=40.50% 
[Step 16] loss=1.91

[Step 1] loss=3.69e+01 train accu=32.50% validation accu=48.50% 
[Step 2] loss=2.58e+01 train accu=61.00% validation accu=50.00% 
[Step 3] loss=2.17e+01 train accu=68.50% validation accu=52.50% 
[Step 4] loss=2.00e+01 train accu=70.50% validation accu=54.50% 
[Step 5] loss=1.73e+01 train accu=77.50% validation accu=60.00% 
[Step 6] loss=1.59e+01 train accu=80.00% validation accu=51.50% 
[Step 7] loss=1.56e+01 train accu=78.50% validation accu=60.50% 
[Step 8] loss=1.39e+01 train accu=83.50% validation accu=54.50% 
[Step 9] loss=1.40e+01 train accu=82.00% validation accu=56.00% 
[Step 10] loss=1.32e+01 train accu=84.00% validation accu=58.00% 
[Step 11] loss=1.34e+01 train accu=83.00% validation accu=59.00% 
[Step 12] loss=1.16e+01 train accu=86.00% validation accu=60.00% 
[Step 13] loss=1.10e+01 train accu=86.00% validation accu=54.00% 
[Step 14] loss=1.04e+01 train accu=88.00% validation accu=57.50% 
[Step 15] loss=1.01e+01 train accu=86.00% validation accu=57.50% 
[Step 16] loss=9.87

[Step 1] loss=4.11e+01 train accu=26.00% validation accu=31.50% 
[Step 2] loss=3.49e+01 train accu=38.50% validation accu=43.00% 
[Step 3] loss=3.18e+01 train accu=47.00% validation accu=45.00% 
[Step 4] loss=2.90e+01 train accu=53.00% validation accu=49.50% 
[Step 5] loss=2.69e+01 train accu=62.00% validation accu=44.00% 
[Step 6] loss=2.68e+01 train accu=57.00% validation accu=44.50% 
[Step 7] loss=2.50e+01 train accu=61.50% validation accu=44.50% 
[Step 8] loss=2.46e+01 train accu=59.50% validation accu=55.50% 
[Step 9] loss=2.27e+01 train accu=69.50% validation accu=54.00% 
[Step 10] loss=2.21e+01 train accu=65.00% validation accu=51.00% 
[Step 11] loss=2.09e+01 train accu=67.00% validation accu=48.50% 
[Step 12] loss=2.01e+01 train accu=76.50% validation accu=52.50% 
[Step 13] loss=1.99e+01 train accu=71.50% validation accu=51.50% 
[Step 14] loss=1.97e+01 train accu=71.50% validation accu=56.50% 
[Step 15] loss=1.86e+01 train accu=78.00% validation accu=55.00% 
[Step 16] loss=1.69

[Step 1] loss=4.10e+01 train accu=24.50% validation accu=34.00% 
[Step 2] loss=3.25e+01 train accu=44.00% validation accu=45.50% 
[Step 3] loss=2.93e+01 train accu=51.50% validation accu=45.00% 
[Step 4] loss=2.83e+01 train accu=51.50% validation accu=43.00% 
[Step 5] loss=2.67e+01 train accu=57.00% validation accu=50.50% 
[Step 6] loss=2.42e+01 train accu=61.00% validation accu=56.50% 
[Step 7] loss=2.24e+01 train accu=66.50% validation accu=49.50% 
[Step 8] loss=2.19e+01 train accu=67.50% validation accu=50.50% 
[Step 9] loss=2.13e+01 train accu=70.50% validation accu=56.50% 
[Step 10] loss=1.94e+01 train accu=74.00% validation accu=56.50% 
[Step 11] loss=1.97e+01 train accu=74.00% validation accu=53.00% 
[Step 12] loss=1.81e+01 train accu=76.00% validation accu=56.00% 
[Step 13] loss=1.73e+01 train accu=79.50% validation accu=54.50% 
[Step 14] loss=1.73e+01 train accu=80.00% validation accu=57.00% 
[Step 15] loss=1.71e+01 train accu=76.50% validation accu=50.00% 
[Step 16] loss=1.57

[Step 1] loss=4.54e+01 train accu=20.50% validation accu=37.00% 
[Step 2] loss=3.71e+01 train accu=34.50% validation accu=38.00% 
[Step 3] loss=3.39e+01 train accu=43.00% validation accu=38.50% 
[Step 4] loss=3.23e+01 train accu=49.00% validation accu=36.50% 
[Step 5] loss=3.21e+01 train accu=43.50% validation accu=40.50% 
[Step 6] loss=2.98e+01 train accu=51.50% validation accu=38.50% 
[Step 7] loss=2.83e+01 train accu=57.50% validation accu=42.00% 
[Step 8] loss=2.76e+01 train accu=58.00% validation accu=38.00% 
[Step 9] loss=2.90e+01 train accu=56.50% validation accu=44.50% 
[Step 10] loss=2.65e+01 train accu=59.00% validation accu=41.50% 
[Step 11] loss=2.52e+01 train accu=63.50% validation accu=48.00% 
[Step 12] loss=2.36e+01 train accu=70.50% validation accu=48.50% 
[Step 13] loss=2.31e+01 train accu=68.00% validation accu=51.50% 
[Step 14] loss=2.27e+01 train accu=68.00% validation accu=45.00% 
[Step 15] loss=2.12e+01 train accu=74.00% validation accu=44.00% 
[Step 16] loss=2.16

In [112]:
sub_accuracies # subject1 and subject2 were calculated separately, their accuracies are in the spreadsheet 

{'subject1': 0,
 'subject2': 0,
 'subject3': 0.535,
 'subject4': 0.605,
 'subject5': 0.48,
 'subject6': 0.735,
 'subject7': 0.705,
 'subject8': 0.475,
 'subject9': 0.535,
 'subject10': 0.565,
 'subject11': 0.45,
 'subject12': 0.54,
 'subject13': 0.63,
 'subject14': 0.485,
 'subject15': 0.64,
 'subject16': 0.54,
 'subject17': 0.65,
 'subject18': 0.485,
 'subject19': 0.565,
 'subject20': 0.515,
 'subject21': 0.54,
 'subject22': 0.695,
 'subject23': 0.565,
 'subject24': 0.535,
 'subject25': 0.5,
 'subject26': 0.515,
 'subject27': 0.545,
 'subject28': 0.635,
 'subject29': 0.51,
 'subject30': 0.635,
 'subject31': 0.52,
 'subject32': 0.47,
 'subject33': 0.665,
 'subject34': 0.57,
 'subject35': 0.485,
 'subject36': 0.51,
 'subject37': 0.5,
 'subject38': 0.41,
 'subject39': 0.605,
 'subject40': 0.58,
 'subject41': 0.57,
 'subject42': 0.515}

In [118]:
sub_accs_df = pd.DataFrame(list(sub_accuracies.items()), columns=['subject', 'accuracy'])
sub_accs_df.to_csv('subject_dependent_trans.csv')

In [ ]:
## with fine tuning 

In [ ]:
'''
sub_accuracies = {subj: 0 for subj in subjects}

for subj in subjects:
    if subj == 'subject1' or subj == 'subject2':
        continue

    model = AutoModelForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

    target_sampling_rate = 16000
    test_features_list = []
    train_features_list = []
    test_labels_list = [] 
    train_labels_list = []  
    
    num_classes = 5
    
    # dict to store number of emotions stored 
    em_counts = {emotion: 0 for emotion in emotion_to_index.keys()}
    
    for path, sub, em in zip(df.Path, df.Subject, df.Emotion):
        if sub == subj: # subj iterates though each subject 
            waveform, sampling_rate = torchaudio.load(path)
        
            resampler = Resample(orig_freq=sampling_rate, new_freq=target_sampling_rate)
            resampled_waveform = resampler(waveform)
                
            waveform = waveform.squeeze().numpy()
            resampled_waveform = resampled_waveform.squeeze().numpy()
    
            split = divide_into_segments(resampled_waveform, target_sampling_rate) # list of 4 torch tensors 
    
            for i in range(len(split)):
                inputs = feature_extractor(split[i], sampling_rate=target_sampling_rate, padding='max_length', return_tensors='pt')
                input_values = inputs.input_values
    
                label_idx = emotion_to_index[em]
                label = emotion_to_index[em]
    
                if em_counts[em] < 10: 
                    train_features_list.append(input_values)
                    train_labels_list.append(label)
                else:
                    test_features_list.append(input_values)
                    test_labels_list.append(label)
    
            em_counts[em] += 1 
            
        else:
            continue

    test_features = torch.cat(test_features_list, dim=0)
    train_features = torch.cat(train_features_list, dim=0)
    test_labels = torch.Tensor(test_labels_list)
    train_labels = torch.Tensor(train_labels_list)

    train_dataset = TensorDataset(train_features, train_labels)
    test_dataset = TensorDataset(test_features, test_labels)
    loader_train = DataLoader(train_dataset, batch_size=8, shuffle=True)
    loader_test = DataLoader(test_dataset, batch_size=8, shuffle=True)

    model.classifier.dense = torch.nn.Linear(model.classifier.dense.in_features, 5)
    
    for param in model.parameters():
        param.requires_grad = False
    
    for param in model.classifier.dense.parameters():
        param.requires_grad = True

    # weight_decay
    skip = {}
    if hasattr(model, 'no_weight_decay'):
        skip = model.no_weight_decay()
    
    parameters = add_weight_decay(model, 0.0001, skip)
    weight_decay = 0.
    
    optimizer = optim.AdamW(parameters)
    loss_fn = nn.CrossEntropyLoss()

    iters = 20
    train_len, test_len = len(train_dataset), len(test_dataset)
    pb_train = display(progress(0, iters), display_id=True)
    losses = [[]]
    accus_train = [[]]
    accus_val = []
    
    model = model.to(device)
    
    for i in range(iters):
      for x, t in loader_train:
        x, t = x.to(device), t.long().to(device)
        #x, t = x.to(device), t.to(device).long() # comment this line if use option1 dataloader
        logits = model(x).logits
        #print(logits)
        # print(logits.mean())
        loss = loss_fn(logits, t)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses[-1].append(loss.item())
    
        with torch.no_grad():
          accus_train[-1].append(torch.sum(torch.max(logits, dim=1)[1] == t))
          #print(torch.max(logits, dim=1)[1])
        # break
    
      losses[-1] = sum(losses[-1])
      losses.append([])
      accus_train[-1] = sum(accus_train[-1]) / train_len
      accus_train.append([])
    
      # step LR for next epoch
      #lr_scheduler.step(i + 1)
    
      accus_val.append(eval_corona(model, loader_test))
      model.train()
    
      pb_train.update(progress(i, iters))
      print(f'\r[Step {i+1}] loss={loss:.2e} '
            f'\r[Step {i+1}] loss={losses[-2]:.2e} '
            f'train accu={accus_train[-2]:.2%} '
            f'validation accu={accus_val[-1]:.2%} '
            # , end=''
            , flush=True)
      # break

    for param in model.parameters():
    param.requires_grad = True
    
    for param in model.classifier.dense.parameters():
        param.requires_grad = True
        
    lr = 0.00001 
    parameters = add_weight_decay(model, 0.0001, skip)
    optimizer = optim.AdamW(parameters)
    loss_fn = nn.CrossEntropyLoss()
    iters = 10

    pb_train = display(progress(0, iters), display_id=True)
    losses = [[]]
    accus_train = [[]]
    accus_val = []
    max_val = -1
    save_path = f"audioT{subj}_dep.pth"
    
    for i in range(iters):
      for x, t in loader_train:
        x, t = x.to(device), t.long().to(device)
        #print(x)
        logits = model(x).logits
        # print(logits.mean())
        loss = loss_fn(logits, t)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses[-1].append(loss.item())
    
        with torch.no_grad():
          accus_train[-1].append(torch.sum(torch.max(logits, dim=1)[1] == t))
          #print(torch.max(logits, dim=1)[1])
        # break
    
      losses[-1] = sum(losses[-1])
      losses.append([])
      accus_train[-1] = sum(accus_train[-1]) / train_len
      accus_train.append([])
    
      accus_val.append(eval_corona(model, loader_test))
      cur_val_acc = accus_val[-1]
      if cur_val_acc > max_val:
        max_val = cur_val_acc
        torch.save(model, save_path)
    
      model.train()
    
      pb_train.update(progress(i, iters))
      print(f'\r[Step {i+1}] loss={loss:.2e} '
            f'\r[Step {i+1}] loss={losses[-2]:.2e} '
            f'train accu={accus_train[-2]:.2%} '
            f'validation accu={accus_val[-1]:.2%} '
            # , end=''
            , flush=True)
      # break
    model = torch.load(save_path)
    model = model.to(device)
    model.eval()
    
    correct = 0
    total = 0
    
    with torch.no_grad():
      for x, t in loader_test:
        x, t = x.to(device), t.long().to(device)
    
        logits = model(x).logits
        predicted = torch.max(logits, dim=1)[1]
    
        total += t.size(0)
        correct += (predicted==t).sum().item()
    
    finetune_acc = correct/total
    print(f'Test set accuracy: {finetune_acc*100:.3f}%')
    sub_accuracies[subj] = finetune_acc
'''